In [ ]:
import pandas as pd
import pm4py

from pm4py.objects.conversion.log import converter as log_converter
from pm4py.visualization.petri_net import visualizer as pn_visualizer
from pm4py.visualization.bpmn import visualizer as bpmn_visualizer
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
import pm4py.convert as conv

import sys

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')
sys.path.insert(0, '../../../../..')

In [ ]:
helpdesk_log_path = "../../../../probabilistic_suffix_prediction_dev/data/helpdesk.csv"  

sepsis_log_path = "../../../../../evaluation_results/data/Sepsis.csv" 

In [ ]:
df = pd.read_csv(helpdesk_log_path)

new_df = pd.DataFrame(columns=['case:concept:name', 'concept:name', 'time:timestamp'])

new_df["case:concept:name"] = df["CaseID"].astype(str)
new_df["concept:name"] = df["Activity"].astype(str)

# Optional: Ensure timestamp is parsed correctly
df["CompleteTimestamp"] = pd.to_datetime(df["CompleteTimestamp"])
new_df['time:timestamp'] = pd.to_datetime(df["CompleteTimestamp"])

print(new_df.head(10))

# Step 2: Convert to PM4Py EventLog object
event_log = log_converter.apply(new_df, variant=log_converter.Variants.TO_EVENT_LOG)

print(type(event_log))

# Step 3: Apply Inductive Miner
net, im, fm = pm4py.discover_petri_net_inductive(
    event_log,
    multi_processing=False,
    noise_threshold=0.0,
    activity_key="concept:name",
    timestamp_key="time:timestamp",
    case_id_key="case:concept:name",
    disable_fallthroughs=False
)

# Step 4: Visualize Petri net
gviz = pn_visualizer.apply(net, im, fm)
pn_visualizer.view(gviz)

tree = inductive_miner.apply(
    event_log,
    parameters={"noise_threshold": 1.0}
)

bpmn = conv.convert_to_bpmn(tree)
gviz_bpmn = bpmn_visualizer.apply(bpmn)
bpmn_visualizer.view(gviz_bpmn)


In [ ]:
df = pd.read_csv(sepsis_log_path)

new_df = pd.DataFrame(columns=['case:concept:name', 'concept:name', 'time:timestamp'])

new_df["case:concept:name"] = df["case:concept:name"].astype(str)
new_df["concept:name"] = df["concept:name"].astype(str)

# Optional: Ensure timestamp is parsed correctly
df["time:timestamp"] = pd.to_datetime(df["time:timestamp"])
new_df['time:timestamp'] = pd.to_datetime(df["time:timestamp"])

print(new_df.head(10))

# Step 2: Convert to PM4Py EventLog object
event_log = log_converter.apply(new_df, variant=log_converter.Variants.TO_EVENT_LOG)

print(type(event_log))

# Step 3: Apply Inductive Miner
net, im, fm = pm4py.discover_petri_net_inductive(
    event_log,
    multi_processing=False,
    noise_threshold=0.0,
    activity_key="concept:name",
    timestamp_key="time:timestamp",
    case_id_key="case:concept:name",
    disable_fallthroughs=False
)

# Step 4: Visualize Petri net
gviz = pn_visualizer.apply(net, im, fm)
pn_visualizer.view(gviz)

tree = inductive_miner.apply(
    event_log,
    parameters={"noise_threshold": 0.0}
)

bpmn = conv.convert_to_bpmn(tree)
gviz_bpmn = bpmn_visualizer.apply(bpmn)
bpmn_visualizer.view(gviz_bpmn)